# Пространства имен [Namespaces]

### Четыре пространства имён в питоне

1. Built-In
2. Global
3. Enclosing
4. Local

### Built-In (встроенный namespace)

In [ ]:
print(dir(__builtins__))

### Когда создается и сколько экземпляров

* Создается в момент запуска скрипта и удаляется в момент завершения скрипта
* Один на всю программу

### Global (глобальный namespace)

In [ ]:
x = 1
print(globals())

### Находим несохраненный результат из блока

In [ ]:
sum([1, 2, 3])

In [ ]:
print(globals())

In [ ]:
_3

### Когда создается и сколько экземпляров

* Создается в момент запуска скрипта и удаляется в момент завершения скрипта
* У каждого модуля свой global namespace

### Enclosing and local namespaces

In [ ]:
def f():
    x = 1
    def g():
        x = 2
        print(x)
    g()
    print(x)
f()

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование вложенных функций
</div>

In [ ]:
def f(x):
    y = 1
    print(locals())
f(10)

### Функции создают свой namespace

In [ ]:
def function():
    inner_variable = 42

function()
inner_variable

In [ ]:
def function(arg):
    print(locals())
    print(locals() == globals())

function(1)

In [ ]:
locals() == globals()

### Циклы и условия не создают свой namespace

In [ ]:
for i in range(3):
    in_for = i

print(in_for)
print(i)

In [ ]:
if True:
    in_if = 2
    
print(in_if)

### Генераторы создают namespace

In [ ]:
i = 'Hello'
[i for i in range(10)]
print(i)

### Variable Scope & LEGB rule

1. Local
2. Enclosing
3. Global
4. Built-in

In [ ]:
global_var = 'global_var'

def func(): 
    local_var = 'local_var'
    # global_var is in global namespace
    print('func:', global_var)
    print('func:', local_var)

func()
print(global_var)
print(local_var)

In [ ]:
from numpy import sum
sum.__doc__

### Вложенные функции

In [ ]:
def outer():
    outer_var = 'foo'

    def inner():
        inner_var = 'bar'
        print('from inner:', outer_var)
        print('from inner:', inner_var)

    inner()

    print('from outer:', outer_var)
    print('from outer:', inner_var)

outer()

### Замечание
Функции имеют доступ к внешним пространствам имён относительно того места где они были **определены**, а не **вызваны**

In [ ]:
def f():
    print(it)

def q(func):
    for it in range(10):
        func()
    print(it)

q(f)

### Когда создется объект на запись?

In [ ]:
global_var = 'global_var'

def func():
    global_var = 'global_var_modified'
    # global_var shadows another variable with same name
    print('func  :', global_var)

func()
print('global:', global_var)

### global 

In [ ]:
global_var = 'global_var'

def func():
    global global_var
    global_var = 'global_var_modified'
    print(global_var)

func()
print(global_var)

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование global
</div>

In [ ]:
import dis
global_var = 'global_var'

def func():
    global global_var
    global_var = 'global_var_modified'
    print(global_var)

func()
print(global_var)
dis.dis(func)

In [ ]:
def func():
    # global can create
    global new_global_var
    new_global_var = 'new_global_var'
    print(new_global_var)

func()
print(new_global_var)

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование global
</div>

### Проблема

In [ ]:
def outer():
    var = 'outer'

    def inner():
        global var
        var = 'inner'
        print('from inner:', var)

    inner()
    print('from outer:', var)

outer()
print('from global:', var)

In [ ]:
# cleanup
del var

### Решение : nonlocal

In [ ]:
def outer():
    var = 'outer'

    def inner():
        nonlocal var
        var = 'inner'
        print('from inner :', var)

    inner()    
    print('from outer :', var)

outer()
print('from global:', var)

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование nonlocal
</div>

In [ ]:
# Partial reuse of enclosing variables
variable = 'global value'

def foo(variable, flag=False):
    def bar():
        # nonlocal variable
        if flag:
            variable = 'value'
        print(variable)
    return bar()

foo(variable, flag=True)
foo(variable, flag=False)

### Странное решение

In [ ]:
# without nonlocal

def outer(): 
    outer.var = 'v1'

    def inner():
        outer.var = 'v2'
        print('from inner :', outer.var)

    inner()
    print('from outer :', outer.var)


outer()
print('from global:', outer.var)

### Где теперь лежит переменная?

In [ ]:
def outer():
    enclosing_var = 'outer'

    def inner():
        print('from inner :', enclosing_var)

    return inner

enclosing_var = 'global'
f = outer()
f()
print('from global:', enclosing_var)

## Замыкания [Closures]
_In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined._

Существования замыканий следует из правила LEGB и возможностью оперировать с функциями как обьектами.

In [ ]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

In [ ]:
add_two  = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

In [ ]:
make_adder(2)(7)

<div class="alert alert-danger">
<b>Антипаттерн: </b> использование сложных замыканий
</div>

In [ ]:
def value_factory(value=10):
    def get_value():
        return value

    def set_value(new_value):
        nonlocal value
        value = new_value
        return value
    return get_value, set_value


get_value, set_value = value_factory()
print(get_value())

set_value(10**6)

print(get_value())

In [ ]:
def make_adder(x):

    def add(y):
        return x + y

    def update(new_x):
        nonlocal x
        x = new_x

    add.update = update
    return add


adder = make_adder(10)
print(adder(10))

adder.update(100)

print(adder(10))


## Декораторы

In [ ]:
import sys

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return inner


pprint = deprecate(print)

pprint([1, 2, 3])

### Синтаксис декораторов

In [ ]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def add(x, y):
    return x + y

add(1, 2)

### Pusheenize

In [ ]:
from IPython import display

def pusheenize(func):
    return display.HTML('<img src="https://media1.tenor.com/images/4a950a1e221d93e654047ecee711af5a/tenor.gif">')

@pusheenize
def dummy_function(arg):
    print(arg)

dummy_function

### Проблема

In [ ]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

### Решение 1

In [5]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Решение 2

In [3]:
import functools

def deprecated(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Декоратор Once

In [2]:
def once(func):
    called = False

    def wrapper(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return func(*args, **kwargs)

    return wrapper


@once
def f():
    print('Hi!')


f()
f()
f()

Hi!


![dicaprio](https://camo.githubusercontent.com/68cfb12d9c144fec645a61fa64151b35efc70f68/687474703a2f2f69302e6b796d2d63646e2e636f6d2f70686f746f732f696d616765732f6f726967696e616c2f3030302f3338342f3137362f6432662e6a7067)

In [4]:
import functools
import sys

def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(
                f'{func.__name__} called with args {args}, {kwargs}!',
                file=dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace()
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

f called with args ('Hi!',), {'test': 42}!
f called with args ('Hi!', 21.0), {}!
f called with args ('Hi!', 10.5), {}!
f called with args ('Hi!', 5.25), {}!
f called with args ('Hi!', 2.625), {}!
f called with args ('Hi!', 1.3125), {}!
f called with args ('Hi!', 0.65625), {}!


### Цепочки декораторов

In [ ]:
@deprecated
@trace()
def f(x):
    return x

f(1)

# Декоратор notify

In [ ]:
# notify: decorator
# - без параметров: отправляет сообщение в чат
# - с параметрами: отправляет сообщение с переданными параметрами
import functools
from telegram_api import send


def notify(*dec_args):
    def wrapper(func):
        @functools.wraps(func)
        def wraps(*args, **kwargs):
            r = func(*args, **kwargs)
            # your code goes here
            return r
        return wraps
    return wrapper


@notify
def foo():
    '''Docstring'''
    print('foo is done')


@notify('hello chat')
def bar():
    print('bar is done')


foo()
bar()